<a href="https://colab.research.google.com/github/Cypher29/AI_repo/blob/main/HGF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install python-dotenv
#pip install anthropic

In [1]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

# New Section

In [ ]:
# if this gives an "ERROR" about pip dependency conflicts, ignore it! It doesn't affect anything.
!pip install -q transformers datasets diffusers

Gradio is for quick User Inerface design

In [3]:
pip install --upgrade gradio

In [5]:
import torch
from google.colab import userdata
from huggingface_hub import login
from transformers import pipeline
from diffusers import DiffusionPipeline
from datasets import load_dataset
import soundfile as sf
from IPython.display import Audio
import gradio as gr
import os
#import anthropic

In [6]:
hf_token = userdata.get('HUGGINGFACE_KEY')
login(hf_token, add_to_git_credential=True)

In [8]:
# Sentiment Analysis

#classifier = pipeline("sentiment-analysis", device="cuda")
classifier = pipeline("sentiment-analysis")
result = classifier("LLM's are very helpful")
print(result)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9995296001434326}]


In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

inputs = tokenizer("Welcome to mysteries of LLM", return_tensors="pt")
print(inputs)
len(inputs)
with torch.no_grad():
    logits = model(**inputs).logits
print(logits)
predicted_class_id = logits.argmax().item()
print(predicted_class_id)
model.config.id2label[predicted_class_id]

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english', trust_remote_code=True)
text = "Welcome to mysteries of LLM"
tokens = tokenizer.encode(text)
print(tokens)
tokenizer.decode(tokens)
tokenizer.batch_decode(tokens)
tokenizer.get_added_vocab()



In [ ]:
from google import genai

In [ ]:
GOOGLE_token = userdata.get('GOOGLE_KEY')
client = genai.Client(api_key=GOOGLE_token)

In [ ]:
response = client.models.generate_content(
    model="gemini-2.0-flash", contents="Hi there"
)
print(response.text)

In [ ]:
import gradio as gr

In [ ]:
def greet(name, intensity):
    return "Hello, " + name + "!" * int(intensity)

demo = gr.Interface(
    fn=greet,
    inputs=["text", "slider"],
    outputs=["text"],
)

In [ ]:
demo.launch()

In [ ]:
# Named Entity Recognition

ner = pipeline("ner", grouped_entities=True, device="cuda")
result = ner("Modi is the prime minister of India")
print(result)

In [ ]:
# Question Answering with Context

question_answerer = pipeline("question-answering", device="cuda")
result = question_answerer(question="Who is prime minister of India?", context="Modi is the prime minister of India. He is from BJP government")
print(result)

In [ ]:
# Text Summarization

summarizer = pipeline("summarization", device="cuda")
text = """The Hugging Face transformers library is an incredibly versatile and powerful tool for natural language processing (NLP).
It allows users to perform a wide range of tasks such as text classification, named entity recognition, and question answering, among others.
It's an extremely popular library that's widely used by the open-source data science community.
It lowers the barrier to entry into the field by providing Data Scientists with a productive, convenient way to work with transformer models.
"""
summary = summarizer(text, max_length=50, min_length=25, do_sample=False)
print(summary[0]['summary_text'])

In [ ]:
# Translation

translator = pipeline("translation_en_to_fr", device="cuda")
result = translator("The Data Scientists were truly amazed by the power and simplicity of the HuggingFace pipeline API.")
print(result[0]['translation_text'])

In [ ]:
# Another translation, showing a model being specified
# All translation models are here: https://huggingface.co/models?pipeline_tag=translation&sort=trending

translator = pipeline("translation_en_to_es", model="Helsinki-NLP/opus-mt-en-es", device="cuda")
result = translator("The Data Scientists were truly amazed by the power and simplicity of the HuggingFace pipeline API.")
print(result[0]['translation_text'])

In [ ]:
# Classification

classifier = pipeline("zero-shot-classification", device="cuda")
result = classifier("Hugging Face's Transformers library is amazing!", candidate_labels=["technology", "sports", "politics"])
print(result)

In [ ]:
# Text Generation

generator = pipeline("text-generation", device="cuda")
result = generator("In bodybuilding biceps training is easy also abs training")
print(result[0]['generated_text'])

In [ ]:
# Image Generation

image_gen = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2",
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16"
    ).to("cuda")

text = "a pluffy smily wolf which is happy"
image = image_gen(prompt=text).images[0]
image

In [ ]:
# Audio Generation

synthesiser = pipeline("text-to-speech", "microsoft/speecht5_tts", device='cuda')

embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embedding = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

speech = synthesiser("Hi daksha, how are you today. ", forward_params={"speaker_embeddings": speaker_embedding})

sf.write("speech.wav", speech["audio"], samplerate=speech["sampling_rate"])
Audio("speech.wav")

In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1")
model.to("cuda")

generated_ids = model.generate(tokens, max_new_tokens=1000, do_sample=True)

# decode with mistral tokenizer
result = tokenizer.decode(generated_ids[0].tolist())
print(result)

Open AI

In [9]:
from openai import OpenAI
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_KEY')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [10]:
system_message = "You are an assistant that is great at telling kids stories"
user_prompt = "Tell a small story about animals in forrest"

prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [ ]:
completion = openai.chat.completions.create(model='gpt-3.5-turbo', messages=prompts)
print(completion.choices[0].message.content)

In [ ]:
completion = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.4
)

print(completion.choices[0].message.content)

In [ ]:
len(completion.choices)
print(completion)

In [14]:
def mychat(text):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": text}
      ]
    completion  = openai.chat.completions.create(
    model='gpt-4o',
    messages=messages,
    temperature=0.4
    )
    print(completion)
 #   result = ""
 #   for chunk in stream:
 #      result += chunk.choices[0].delta.content or ""
 #      print(result)
 #      yield result
    return completion.choices[0].message.content
#gr.Interface(fn=shout, inputs="textbox", outputs="textbox").launch()

In [15]:
mychat('hi there')

ChatCompletion(id='chatcmpl-BU1HAcFrPec4Gk0O4fv9ozOTMpVpQ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today? Would you like to hear a story or perhaps create one together?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1746494444, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_d8864f8b6b', usage=CompletionUsage(completion_tokens=23, prompt_tokens=24, total_tokens=47, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


'Hello! How can I assist you today? Would you like to hear a story or perhaps create one together?'

In [16]:
system_message = "You are a helpful assistant that responds in markdown"
view = gr.Interface(
    fn=mychat,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=6)],
    flagging_mode="never"
)
view.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://caf536c52d91884546.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [17]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    completion  = openai.chat.completions.create(
    model='gpt-4o',
    messages=messages,
    temperature=0.4
    )

    return completion.choices[0].message.content

**Agent**

In [18]:
system_message = "You are a helpful assistant for a bookshop. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [19]:
book_prices = {"Two Cities": "450", "Prince": "999", "The Hobbit": "750", "Dune": "500"}

def get_book_price(bookName):
    #pdb.set_trace()
    print(f"Tool get_book_price called for {bookName}")
    book = bookName.lower()
    return book_prices.get(book, "Unknown")

In [ ]:
get_ticket_price("London")

In [20]:
price_function = {
    "name": "get_book_price",
    "description": "Get the price of a book. Call this whenever you need to know the book price, for example when a customer asks 'How much is this book'",
    "parameters": {
        "type": "object",
        "properties": {
            "bookName": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["bookName"],
        "additionalProperties": False
    }
}

In [21]:
tools = [{"type": "function", "function": price_function}]

In [22]:
import pdb
def chat(message, history):
  #  if(history == None):
   # history = [{'role': 'user', 'metadata': None, 'content': 'hello', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'yes', 'options': None}]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    #pdb.set_trace()
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    #pdb.set_trace()
    return response.choices[0].message.content

In [23]:
import json
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    book = arguments.get('destination_city')
    price = get_book_price(book)
    response = {
        "role": "tool",
        "content": json.dumps({"bookName": book,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [24]:
chat('Prince', [])

Tool get_book_price called for None


AttributeError: 'NoneType' object has no attribute 'lower'

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()


In [ ]:

def echo(message, history):
    if(history == None):
        history = []

    return message+" "+str(history)



In [ ]:
demo = gr.ChatInterface(fn=echo, type="messages", examples=["hello", "hola", "merhaba"], title="Echo Bot")
demo.launch()

Using **Claude**

In [ ]:
claude_key = userdata.get('CLAUDE_KEY')
claude = anthropic.Anthropic(
    api_key=claude_key,
)
#claude = anthropic.Anthropic()
system_message = "You are a helpful assistant"

In [ ]:
def stream_claude(prompt):
    result = claude.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    return result.completion

In [ ]:
print(stream_claude("Hi there"))
